# ETL Pipelines for Data Science Workflows

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from datetime import datetime
from io import StringIO

In [2]:
def create_sample_csv_data():
    """Create sample data and save as CSV to simulate downloading from a source"""
    np.random.seed(42)
    data = {
        'transaction_id': [f'TXN_{i:05d}' for i in range(1000)],
        'customer_id': np.random.randint(1, 201, 1000),
        'product_name': np.random.choice(['Laptop', 'Phone', 'Tablet', 'Headphones', 'Mouse'], 1000),
        'price': np.round(np.random.uniform(10, 1000, 1000), 2),
        'quantity': np.random.randint(1, 6, 1000),
        'transaction_date': pd.date_range('2024-01-01', periods=1000, freq='h'),
        'customer_email': [f'user{i}@email.com' if i % 10 != 0 else None for i in range(1000)]
    }
    df = pd.DataFrame(data)
    df.to_csv('raw_transactions.csv', index=False)
    print("Sample CSV file 'raw_transactions.csv' created!")
    return 'raw_transactions.csv'


## Step 1: Extract

In [3]:
def extract_data_from_csv(csv_file_path):
    """Extract: Load data from CSV file (simulating download from external source)"""
    try:
        print(f"Extracting data from {csv_file_path}...")
        df = pd.read_csv(csv_file_path)
        print(f"Successfully extracted {len(df)} records")
        return df
    except FileNotFoundError:
        print(f"Error: {csv_file_path} not found. Creating sample data...")
        csv_file = create_sample_csv_data()
        return pd.read_csv(csv_file)

## Step 2: Transform

In [4]:
def transform_data(df):
    """Transform: Clean and enrich the data"""
    print("Transforming data...")
    
    # Start with a copy to avoid modifying original
    df_clean = df.copy()
    
    # Remove records with missing emails (data quality)
    initial_count = len(df_clean)
    df_clean = df_clean.dropna(subset=['customer_email'])
    removed_count = initial_count - len(df_clean)
    print(f"Removed {removed_count} records with missing emails")
    
    # Calculate derived fields
    df_clean['total_amount'] = df_clean['price'] * df_clean['quantity']
    
    # Extract date components for better analysis
    df_clean['transaction_date'] = pd.to_datetime(df_clean['transaction_date'])
    df_clean['year'] = df_clean['transaction_date'].dt.year
    df_clean['month'] = df_clean['transaction_date'].dt.month
    df_clean['day_of_week'] = df_clean['transaction_date'].dt.day_name()
    
    # Create customer segments based on spending
    df_clean['customer_segment'] = pd.cut(df_clean['total_amount'], 
                                        bins=[0, 50, 200, float('inf')], 
                                        labels=['Low', 'Medium', 'High'])
    
    print(f"Transformation complete. {len(df_clean)} clean records ready")
    return df_clean

## Step 3: Load

In [5]:
def load_data_to_sqlite(df, db_name='ecommerce_data.db', table_name='transactions'):
    """Load: Save transformed data to SQLite database"""
    print(f"Loading data to SQLite database '{db_name}'...")
    
    # Connect to SQLite database (creates if doesn't exist)
    conn = sqlite3.connect(db_name)
    
    try:
        # Load data to database
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        
        # Verify the load was successful
        cursor = conn.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        record_count = cursor.fetchone()[0]
        
        print(f"Successfully loaded {record_count} records to '{table_name}' table")
        
        # Show a sample of what was loaded
        print("\nSample of loaded data:")
        sample_df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", conn)
        print(sample_df.to_string(index=False))
        
        return f"Data successfully loaded to {db_name}"
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
        
    finally:
        conn.close()

## Running the ETL Pipeline

In [6]:
def run_etl_pipeline():
    """Execute the complete ETL pipeline"""
    print("Starting ETL Pipeline...")
    print("=" * 50)
    
    # Extract
    raw_data = extract_data_from_csv('raw_transactions.csv')
    
    # Transform
    transformed_data = transform_data(raw_data)
    
    # Load
    load_result = load_data_to_sqlite(transformed_data)
    
    print("=" * 50)
    print("ETL Pipeline completed successfully!")
    
    return transformed_data

In [5]:
create_sample_csv_data()

Sample CSV file 'raw_transactions.csv' created!


'raw_transactions.csv'

In [10]:
run_etl_pipeline()

Starting ETL Pipeline...
Extracting data from raw_transactions.csv...
Successfully extracted 1000 records
Transforming data...
Removed 100 records with missing emails
Transformation complete. 900 clean records ready
Loading data to SQLite database 'ecommerce_data.db'...
Successfully loaded 900 records to 'transactions' table

Sample of loaded data:
transaction_id  customer_id product_name  price  quantity    transaction_date  customer_email  total_amount  year  month day_of_week customer_segment
     TXN_00001          180   Headphones 945.17         2 2024-01-01 01:00:00 user1@email.com       1890.34  2024      1      Monday             High
     TXN_00002           93       Tablet 686.42         1 2024-01-01 02:00:00 user2@email.com        686.42  2024      1      Monday             High
     TXN_00003           15        Phone 502.20         5 2024-01-01 03:00:00 user3@email.com       2511.00  2024      1      Monday             High
     TXN_00004          107       Laptop 621.67  

,transaction_id,customer_id,product_name,price,quantity,transaction_date,customer_email,total_amount,year,month,day_of_week,customer_segment
1,TXN_00001,180,Headphones,945.17,2,2024-01-01 01:00:00,user1@email.com,1890.34,2024,1,Monday,High
2,TXN_00002,93,Tablet,686.42,1,2024-01-01 02:00:00,user2@email.com,686.42,2024,1,Monday,High
3,TXN_00003,15,Phone,502.20,5,2024-01-01 03:00:00,user3@email.com,2511.00,2024,1,Monday,High
4,TXN_00004,107,Laptop,621.67,4,2024-01-01 04:00:00,user4@email.com,2486.68,2024,1,Monday,High
5,TXN_00005,72,Mouse,870.22,5,2024-01-01 05:00:00,user5@email.com,4351.10,2024,1,Monday,High
...,...,...,...,...,...,...,...,...,...,...,...,...
995,TXN_00995,111,Laptop,764.28,2,2024-02-11 11:00:00,user995@email.com,1528.56,2024,2,Sunday,High
996,TXN_00996,111,Mouse,835.31,3,2024-02-11 12:00:00,user996@email.com,2505.93,2024,2,Sunday,High
997,TXN_00997,34,Mouse,446.04,2,2024-02-11 13:00:00,user997@email.com,892.08,2024,2,Sunday,High
998,TXN_00998,111,Phone,309.31,2,2024-02-11 14:00:00,user998@email.com,618.62,2024,2,Sunday,High
